In [1]:
!pip install TTS torchaudio librosa gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 54.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 66.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 106.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.

In [2]:
!mkdir -p voice-cloner-project/data/librispeech_sample
!wget -c https://www.openslr.org/resources/12/dev-clean.tar.gz
!tar -xzf dev-clean.tar.gz --strip-components=2 -C voice-cloner-project/data/librispeech_sample LibriSpeech/dev-clean/84/121123


--2025-07-11 07:27:23--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/12/dev-clean.tar.gz [following]
--2025-07-11 07:27:24--  https://openslr.elda.org/resources/12/dev-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  22.1MB/s    in 17s     

2025-07-11 07:27:42 (18.7 MB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]



In [3]:
import os
import torchaudio
import librosa
import soundfile as sf
from glob import glob

# Convert FLAC to WAV
flac_files = glob('voice-cloner-project/data/librispeech_sample/*.flac')
output_dir = "voice-cloner-project/data/my_wavs"
os.makedirs(output_dir, exist_ok=True)

for file in flac_files:
    wav_out = os.path.join(output_dir, os.path.basename(file).replace(".flac", ".wav"))
    waveform, sr = torchaudio.load(file)
    waveform = waveform.mean(dim=0).unsqueeze(0)  # Convert to mono
    torchaudio.save(wav_out, waveform, 16000)


In [4]:
import os
import torchaudio
import librosa
import soundfile as sf
from glob import glob

# Convert FLAC to WAV
flac_files = glob('voice-cloner-project/data/librispeech_sample/*.flac')
output_dir = "voice-cloner-project/data/my_wavs"
os.makedirs(output_dir, exist_ok=True)

for file in flac_files:
    wav_out = os.path.join(output_dir, os.path.basename(file).replace(".flac", ".wav"))
    waveform, sr = torchaudio.load(file)
    waveform = waveform.mean(dim=0).unsqueeze(0)  # Convert to mono
    torchaudio.save(wav_out, waveform, 16000)


In [6]:
# ✅ Import the Coqui TTS API
from TTS.api import TTS

# ✅ OPTIONAL: To see a list of available models (using class directly, not via instance)
from TTS.utils.manage import ModelManager
manager = ModelManager()
models = manager.list_models()
print("✅ Available models:", models)



ModuleNotFoundError: No module named 'numpy.rec'

In [ ]:
# ✅ Load the model (this is a multilingual, multispeaker model that supports cloning)
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=False)


In [ ]:
from TTS.api import TTS

# ✅ Load a simple English single-speaker model (no espeak, no speakers)
tts = TTS(model_name="tts_models/en/jenny/jenny", progress_bar=False, gpu=False)

# ✅ Generate audio
tts.tts_to_file(
    text="Whispers of the Wind

The wind walks softly through the trees,
Carrying tales from distant seas.
It hums a tune the stars once knew,
And paints the dawn with drops of dew.

A leaf takes flight without a care,
Dancing gently through the air.
The sky blushes with twilight’s flame,
As day and night forget their name.

A moment lost, a moment found,
Where silence sings without a sound.
So let your heart like rivers bend—
The wind begins, but has no end.

s",
    file_path="output.wav"
)


In [ ]:
import IPython.display as ipd
ipd.Audio("output.wav")


In [ ]:
import gradio as gr
from TTS.api import TTS

# ✅ Load single-speaker model (no speaker needed)
tts = TTS(model_name="tts_models/en/jenny/jenny", progress_bar=False, gpu=False)

# ✅ Define app function
def clone_voice(text):
    tts.tts_to_file(text=text, file_path="output.wav")
    return "output.wav"

# ✅ Build Gradio interface
app = gr.Interface(
    fn=clone_voice,
    inputs="text",
    outputs="audio",
    title="🗣️ Uzair rafique chohan Voice Cloner (Single Speaker)"
)

# ✅ Launch the app
app.launch()
